In [1]:
import pandas as pd
import cvxpy as cp
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import linprog
from pulp import *

In [2]:
# Read input from excel sheet
input_data1 = pd.read_excel(r'C:\Users\fenil\OneDrive\Desktop\6990\Input.xlsx', sheet_name="Sheet5").fillna(0)
input_data2 = pd.read_excel(r'C:\Users\fenil\OneDrive\Desktop\6990\Input.xlsx', sheet_name="Sheet6")

In [3]:
j = 10
k = 30
l = 5

# Extract input data
s = input_data1["Strength of Dept"].values
c = int(input_data1.loc[0, "Total number of employees"])
f = int(input_data1.loc[0,"Number of in person days"])
u = int(input_data1.loc[0, "Sum of Priorities"])
P = input_data1["Priority of each Team/Department"].values
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
a = input_data2.values
s = s.reshape(-1,1)
P = P.reshape(-1,1)

In [4]:
input_data1.shape

(30, 6)

In [5]:
input_data2.shape

(10, 30)

In [6]:
input_data2

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1


In [7]:
print("Total number of employees = ", c)
print("Number of in person days = ", f)
print("Sum of Priorities = ", u)

Total number of employees =  130
Number of in person days =  3
Sum of Priorities =  16


In [8]:
print(s.shape)
print(s)

(30, 1)
[[5]
 [6]
 [5]
 [5]
 [2]
 [2]
 [2]
 [3]
 [5]
 [4]
 [5]
 [2]
 [5]
 [3]
 [7]
 [4]
 [3]
 [4]
 [2]
 [6]
 [5]
 [2]
 [7]
 [5]
 [5]
 [4]
 [6]
 [7]
 [2]
 [7]]


In [9]:
print(P.shape)
print(P)

(30, 1)
[[0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]]


In [10]:
print(a.shape)
print(a)

(10, 30)
[[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]]


In [11]:
x = cp.Variable((k, l), boolean=True)
print(x.shape)
print(x)

(30, 5)
var1


In [12]:
objective = cp.Minimize(cp.sum(cp.multiply(s, x)))

constraints = [
    cp.sum(cp.multiply(s, x), axis=0) <= 1.01 * c * f / 5,
    cp.sum(cp.multiply(s, x), axis=0) >= 0.99 * c * f / 5,  
]
for j_ in range(j):
    for l_ in range(l):
        constraints.append(cp.sum(x[np.where(a[j_,:] == 1)[0], l_]) >= 1)

if u == 0:
    constraints.append(cp.sum(x, axis=1) == f)
elif u >= 1:
    for k in range(k):
        if P[k] == 0:
            constraints.append(cp.sum(x[k, :]) <= f)
        elif P[k] == 1:
            constraints.append(cp.sum(x[k, :]) == l)

problem = cp.Problem(objective, constraints)
problem.solve()

# Solve optimization problem and convert result to DataFrame
result = x.value.astype(int)
result = result  # add 1 to team indices
df_result = pd.DataFrame(result, columns=days)
df_result = df_result.rename_axis('Team Number')
df_result.index = df_result.index + 1

team_strength_per_day = df_result.mul(s.flatten(), axis=0)
total_employees_per_day = team_strength_per_day.sum(axis=0)

print(team_strength_per_day)
print(total_employees_per_day)

             Monday  Tuesday  Wednesday  Thursday  Friday
Team Number                                              
1                 0        0          5         0       5
2                 6        6          6         6       6
3                 5        5          0         0       0
4                 0        0          0         0       0
5                 2        2          2         2       2
6                 2        2          2         2       2
7                 0        0          0         0       0
8                 3        3          3         3       3
9                 5        5          5         5       5
10                4        4          4         4       4
11                5        5          5         5       5
12                0        0          0         0       0
13                0        0          0         5       0
14                0        0          3         0       3
15                7        7          0         7       0
16            